# Computational Theory Problems

In [5]:
# Numerical arrays and methods
import numpy as np
import struct
import math
from unittest import result

# Problem 1: Binary Words and Operations
## Overview
Following the Secure Hash Standard PDF, we implement seven bitwise functions that are fundamental to the SHA hashing algorithms.

## Symbols and Operations:

### & (Bitwise AND operation)
Returns 1 only when both of the corresponding bits are 1. If either bit has a 0, the result will be 0.

2-bit Example: 1100 & 1010 = 1000

3-bit Example: 1100 & 1010 & 1110 = 1000

### | (Bitwise OR operation)
Returns 1 only when at least the corresponding bits are 1. It will return 0 when both or all bits are 0.

2-bit Example: 1100 | 1010 = 1110

3-bit Example: 1100 | 1010 | 1110 = 1110

### ^ (Bitwise XOR operation - exclusive-OR)
Returns 1 when an odd number of corresponding bits are 1. It will return 0 when an even number of bits are 1.

2-bit Example: 1100 ^ 1010 = 0110

3-bit Example: 1100 ^ 1010 ^ 1110 = 0110

### ~ (Bitwise complement operation)
Inverts all bits - changes all the 0s to 1s and vice versa. 

2-bit Example: ~1100 = 0011

3-bit Example: ~1100 & ~1010 & ~1110 = 0001

### << (Left-Shift operation)
Ignores the left-most n bits and pads the result with n zeros on the right. It essentially multiplies the bits by 2^n.

2-bit Example: 1100 << 2 = 0000

3-bit Example: 0011 << 3 = 1000

### >> (Right-Shift operation)
Ignores the right-most n bits and pads the result with n zeros on the left. It essentially divides the bits by 2^n.

2-bit Example: 1100 >> 2 = 0011

3-bit Example: 1000 >> 3 = 0001

In [4]:
# Helper functions 
def rotr(x, n):
    """
    Rotate right (circular right shift) the 32-bit integer x by n bits.

    Parameters:
        x (int): The 32-bit integer to rotate.
        n (int): The number of bits/positions to rotate.

    Returns:
        int: The rotated integer as a 32-bit integer.
    """
    x = np.uint32(x)
    return np.uint32((x >> n) | (x << (32 - n)))


def rotl(x, n):
    """
    Rotate left (circular left shift) the 32-bit integer x by n bits.

    Parameters:
        x (int): The 32-bit integer to rotate.
        n (int): The number of bits/positions to rotate.

    Returns:
        int: The rotated integer as a 32-bit integer.
    """
    x = np.uint32(x)
    return np.uint32((x << n) | (x >> (32 - n)))


def shr(x, n):
    """
    A right shift operation only shifts with zeros, no rotating.

    Parameters:
        x (int): The 32-bit integer to shift.
        n (int): The number of bits/positions to shift.

    Returns:
        int: The shifted integer as a 32-bit integer.
    """
    x = np.uint32(x)
    return np.uint32(x >> n)

In [ ]:
# Parity(x, y, z) function

def Parity(x, y, z):
    """
    This function calculates the parity of three 32-bit unsigned integers.

    It will return the result of the bitwise XOR operation on the three inputs.

    Parameters:
        x (int): First 32-bit unsigned integer.
        y (int): Second 32-bit unsigned integer.
        z (int): Third 32-bit unsigned integer.

    Returns:
        int: The parity result as a 32-bit unsigned integer.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    return np.uint32(x ^ y ^ z)

In [11]:
# Ch(x, y, z) Function
def Ch(x, y, z):
   """
   This function calculates the choice function of three 32-bit unsigned integers.

   It will return the result of the bitwise operation: (x AND y) XOR (NOT x AND z).

   Parameters:
         x (int): First 32-bit unsigned integer.
         y (int): Second 32-bit unsigned integer.
         z (int): Third 32-bit unsigned integer.
   Returns:
         int: The choice result as a 32-bit unsigned integer.
   """
   x = np.uint32(x) 
   y = np.uint32(y)
   z = np.uint32(z)
   
   return np.uint32((x & y) ^ (~x & z))

In [12]:
# Maj(x, y, z) Function
def Maj(x, y, z):
    """
    This function calculates the majority function of three 32-bit unsigned integers.

    It will return the result of the bitwise operation: (x AND y) XOR (x AND z) XOR (y AND z).

    Parameters:
        x (int): First 32-bit unsigned integer.
        y (int): Second 32-bit unsigned integer.
        z (int): Third 32-bit unsigned integer.

    Returns:
        int: The majority result as a 32-bit unsigned integer.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    return np.uint32((x & y) ^ (x & z) ^ (y & z))

In [5]:
# Sigma0(x) Function E0
def Sigma0(x):
    """
    This function calculates the Sigma0 (E0) function for a 32-bit unsigned integer.

    It will return the result of the bitwise operations: ROTR(x, 2) XOR ROTR(x, 13) XOR ROTR(x, 22).

    Parameters:
        x (int): A 32-bit unsigned integer.

    Returns:
        int: The Sigma0 result as a 32-bit unsigned integer.
    """
    # Ensure x is treated as a 32-bit unsigned integer
    x = np.uint32(x)
    
    return np.uint32(rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22))


In [6]:
# Sigma1(x) Function E1
def Sigma1(x):
    """
    This function calculates the Sigma1 (E1) function for a 32-bit unsigned integer.

    It will return the result of the bitwise operations: ROTR(x, 6) XOR ROTR(x, 11) XOR ROTR(x, 25).

    Parameters:
        x (int): A 32-bit unsigned integer.

    Returns:
        int: The Sigma1 result as a 32-bit unsigned integer.
    """
    # Ensure x is treated as a 32-bit unsigned integer
    x = np.uint32(x)
    
    return np.uint32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))

In [7]:
# Sigma0(x) Function O0
def sigma0(x):
    """
    This function calculates the sigma0 (o0) function for a 32-bit unsigned integer.

    It will return the result of the bitwise operations: ROTR(x, 7) XOR ROTR(x, 18) XOR SHR(x, 3).

    Parameters:
        x (int): A 32-bit unsigned integer.

    Returns:
        int: The sigma0 result as a 32-bit unsigned integer.
    """
    # Ensure x is treated as a 32-bit unsigned integer
    x = np.uint32(x)
    
    return np.uint32(rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3)) 

In [8]:
# Sigma1(x) Function O1
def sigma1(x):
    """
    This function calculates the sigma1 (o1) function for a 32-bit unsigned integer.

    It will return the result of the bitwise operations: ROTR(x, 17) XOR ROTR(x, 19) XOR SHR(x, 10).

    Parameters:
        x (int): A 32-bit unsigned integer.

    Returns:
        int: The sigma1 result as a 32-bit unsigned integer.
    """
    # Ensure x is treated as a 32-bit unsigned integer
    x = np.uint32(x)
    
    return np.uint32(rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10))

# Problem 2: Fractional Parts of Cube Roots

# Problem 3: Padding

# Problem 4: Hashes

# Problem 5: Passwords


# End